In [1]:
###localization

#belief

p=[]
n=5
for i in range(n):
    p.append(1./n)
print(f"initial belief : {p}")

#next belief

p_hit = 0.6
p_miss = 0.2
p_exact = 0.8
p_overshoot = 0.1
p_undershoot = 0.1
world = ['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'red']
motions = [1, 1, 1]

#the obsolute key function of localisation which is called the measurement update

def sense(p, z):
    q =[]
    
    for i in range(len(p)):
        hit = (world[i] == z)
        q.append(p[i] * (hit * p_hit + (1-hit) * p_miss))
    
    
    return [x/sum(q) for x in q]


def move(p,l):
    q = []
    
    for i in range(len(p)):
        s = p[(i-l) %len(p)] * p_exact
        s = s + p_overshoot * p[(i - l + 1) %len(p)]
        s = s + p_undershoot * p[(i -l -1) % len(p)]
        
        q.append(s)
        
    
    return q
        
            
#for k in range(len(measurements)):
 #   p = sense(p, measurements[k])
    
for i in range(len(measurements)):
    p = sense(p, measurements[i])
    p = move(p, motions[i])
print(p)



 



initial belief : [0.2, 0.2, 0.2, 0.2, 0.2]
[0.0, 0.0, 0.0, 1.0, 0.0]


In [218]:

n = 4
m = 5

p = [[1/n*m for i in range(m)]for j in range(n)]

world = [['green', 'green', 'green'],
         ['green', 'red', 'red'],
         ['green', 'green', 'green']]

world2 = [['red', 'green', 'green', 'red', 'red'],
          ['red', 'red', 'green', 'red', 'red'],
          ['red', 'red', 'green', 'green', 'red'],
          ['red', 'red', 'red', 'red', 'red']]





measurement = ['green','green', 'green', 'green', 'green']
motions = [[0,0], [0,1], [1,0], [1,0], [0,1]]

#measurement = ['red','red']
#motions = [[0,0], [0,1]]

sensor_right = 0.7
p_move = 0.8


def sense(p, z):
    
    q = [[0.0 for i in range(len(p[0]))]for j in range(len(p))]
    q_sum = 0
    for i in range(len(p)):
        for j in range(len(p[0])):
            hit = (world2[i][j] == z)
            q[i][j] = p[i][j] * (hit*sensor_right + (1-hit)*(1-sensor_right))
            q_sum += q[i][j]
        
    
    
    return [[x/q_sum for x in i]for i in q]


def move(p, l):
    q = [[0.0 for i in range(len(p[0]))] for j in range(len(p))]
    
    for i in range(len(p)):
        for j in range(len(p[i])):
            s = p[(i-l[0]) %len(p)][(j-l[1]) %len(p[i])] * p_move
            s =s +(1-p_move) * p[i][j]
            q[i][j] = s
        
    
    return q

for i in range(len(measurement)):
    p = move(p, motions[i])
    p = sense(p, measurement[i])
  
    
    
for i in p:
    print(i)
        
            




[0.011059807427972012, 0.02464041578496803, 0.06799662806785915, 0.044724870458121596, 0.024651531216653727]
[0.007153204183320982, 0.010171326481705892, 0.08696596002664689, 0.07988429965998084, 0.009350668508437186]
[0.007397366886111672, 0.008943730670452702, 0.11272964670259776, 0.35350722955212727, 0.04065549207827676]
[0.009106505805646497, 0.007153204183320982, 0.014349221618346574, 0.04313329135844895, 0.03642559932900475]


In [4]:
###TRACKING

#gaussian
from math import * 
def f(mu, sigma2, x):
    return 1/sqrt(2 * pi * sigma2) * exp((-1.0/2.0) * ((mu - x)**2/sigma2))


print( f(10.0,4.0,10))

def update(mean1, var1, mean2, var2):
    new_mean = (var2 * mean1 + var1 * mean2)/(var1 + var2)
    new_var = 1./(1./var1 + 1./var2)
    return [new_mean, new_var]

def predict(mean1, var1, mean2, var2):
    new_mean = mean1 + mean2
    new_var = var1 + var2
    return [new_mean, new_var]

measurements = [5., 6., 7., 9.,10.]
motion = [1., 1., 2., 1., 1.]
measurement_seg = 4.
motion_seg = 2.
mu = 0.
seg = 1000.

for i in range(len(measurements)):
    mu, seg = update(mu, seg, measurements[i], measurement_seg)
    print(f"[update]::[{mu}, {seg}]")
    mu, seg = predict(mu, seg, motion[i], motion_seg)
    print(f"[predict]::[{mu}, {seg}]")

0.19947114020071635
[update]::[4.9800796812749, 3.9840637450199203]
[predict]::[5.9800796812749, 5.98406374501992]
[update]::[5.992019154030327, 2.3974461292897047]
[predict]::[6.992019154030327, 4.397446129289705]
[update]::[6.996198441360958, 2.094658810112146]
[predict]::[8.996198441360958, 4.094658810112146]
[update]::[8.99812144836331, 2.0233879678767672]
[predict]::[9.99812144836331, 4.023387967876767]
[update]::[9.99906346214631, 2.0058299481392163]
[predict]::[10.99906346214631, 4.005829948139216]


In [4]:
def update(mean1, var1, mean2, var2):
    new_mean = ((var2 * mean1) + (var1 * mean2))/(var2 + var1)
    new_var = 1/(1/var2 + 1/var1)
    return [new_mean, new_var]


def predict(mean1, var1, mean2, var2):
    new_mean = mean1 + mean2
    new_var = var1 + var2
    return [new_mean, new_var]


measurements = [5., 6., 7., 9., 10.]
motion = [1., 1., 2., 1., 1.]
measurement_sig = 4
motion_sig = 2.
mu = 0.
sig = 10000.

for i in range(len(measurements)):
    mu, sig = update(mu, sig, measurements[i], measurement_sig)
    print(f'update: {mu, sig}')
    
    mu, sig = predict(mu, sig, motion[i], motion_sig)
    print(f'predict: {mu, sig}')
    




update: (4.998000799680128, 3.9984006397441023)
predict: (5.998000799680128, 5.998400639744102)
update: (5.999200191953932, 2.399744061425258)
predict: (6.999200191953932, 4.399744061425258)
update: (6.999619127420922, 2.0951800575117594)
predict: (8.999619127420921, 4.09518005751176)
update: (8.999811802788143, 2.0235152416216957)
predict: (9.999811802788143, 4.023515241621696)
update: (9.999906177177365, 2.0058615808441944)
predict: (10.999906177177365, 4.005861580844194)


In [7]:

from math import *

class matrix:
    
    # implements basic operations of a matrix class
    
    def __init__(self, value):
        self.value = value
        self.dimx = len(value)
        self.dimy = len(value[0])
        if value == [[]]:
            self.dimx = 0
    
    def zero(self, dimx, dimy):
        # check if valid dimensions
        if dimx < 1 or dimy < 1:
            raise(ValueError, "Invalid size of matrix")
        else:
            self.dimx = dimx
            self.dimy = dimy
            self.value = [[0 for row in range(dimy)] for col in range(dimx)]
    
    def identity(self, dim):
        # check if valid dimension
        if dim < 1:
            raise (ValueError, "Invalid size of matrix")
        else:
            self.dimx = dim
            self.dimy = dim
            self.value = [[0 for row in range(dim)] for col in range(dim)]
            for i in range(dim):
                self.value[i][i] = 1
    
    def show(self):
        for i in range(self.dimx):
            print(self.value[i])
        print(' ')
    
    def __add__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise (ValueError, "Matrices must be of equal dimensions to add")
        else:
            # add if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] + other.value[i][j]
            return res
    
    def __sub__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise (ValueError, "Matrices must be of equal dimensions to subtract")
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] - other.value[i][j]
            return res
    
    def __mul__(self, other):
        # check if correct dimensions
        if self.dimy != other.dimx:
            raise (ValueError, "Matrices must be m*n and n*p to multiply")
        else:
            # multiply if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, other.dimy)
            for i in range(self.dimx):
                for j in range(other.dimy):
                    for k in range(self.dimy):
                        res.value[i][j] += self.value[i][k] * other.value[k][j]
            return res
    
    def transpose(self):
        # compute transpose
        res = matrix([[]])
        res.zero(self.dimy, self.dimx)
        for i in range(self.dimx):
            for j in range(self.dimy):
                res.value[j][i] = self.value[i][j]
        return res
    
    # Thanks to Ernesto P. Adorio for use of Cholesky and CholeskyInverse functions
    
    def Cholesky(self, ztol=1.0e-5):
        # Computes the upper triangular Cholesky factorization of
        # a positive definite matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        for i in range(self.dimx):
            S = sum([(res.value[k][i])**2 for k in range(i)])
            d = self.value[i][i] - S
            if abs(d) < ztol:
                res.value[i][i] = 0.0
            else:
                if d < 0.0:
                    raise (ValueError, "Matrix not positive-definite")
                res.value[i][i] = sqrt(d)
            for j in range(i+1, self.dimx):
                S = sum([res.value[k][i] * res.value[k][j] for k in range(self.dimx)])
                if abs(S) < ztol:
                    S = 0.0
                try:
                   res.value[i][j] = (self.value[i][j] - S)/res.value[i][i]
                except:
                   raise (ValueError, "Zero diagonal")
        return res
    
    def CholeskyInverse(self):
        # Computes inverse of matrix given its Cholesky upper Triangular
        # decomposition of matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        # Backward step for inverse.
        for j in reversed(range(self.dimx)):
            tjj = self.value[j][j]
            S = sum([self.value[j][k]*res.value[j][k] for k in range(j+1, self.dimx)])
            res.value[j][j] = 1.0/tjj**2 - S/tjj
            for i in reversed(range(j)):
                res.value[j][i] = res.value[i][j] = -sum([self.value[i][k]*res.value[k][j] for k in range(i+1, self.dimx)])/self.value[i][i]
        return res
    
    def inverse(self):
        aux = self.Cholesky()
        res = aux.CholeskyInverse()
        return res
    
    def __repr__(self):
        return repr(self.value)

In [8]:
measurements = [1,2,3]

x = matrix([[0.], [0.]])  #initial state(location and velocity)
p = matrix([[1000., 0.], [0., 1000.]]) #initial uncertainty
u = matrix([[0.], [0.]]) #external motion
f = matrix([[1., 1.], [0., 1.]]) # next state function
h = matrix([[1., 0.]]) # measurement function
r = matrix([[1.]]) # measurement uncertainty
i = matrix([[1., 0.], [0., 1.]]) # identity matrix





In [9]:
# 4 dimensional 

measurements = [[5., 10.], [6., 8.], [7., 6.], [8., 4.], [9., 2.], [10., 0.]]
initial_xy = [4., 12.]
dt = 0.1
x = matrix([[initial_xy[0]], [initial_xy[1]], [0.], [0.]])  #initial state(location and velocity)
p = matrix([[0., 0., 0., 0.], 
            [0., 0., 0., 0.], 
            [0., 0., 1000., 0.], 
            [0., 0., 0., 1000.]]) #initial uncertainty

u = matrix([[0.], [0.], [0.], [0.]]) #external motion
f = matrix([[1., 0., dt, 0.],
            [0., 1., 0., dt],
            [0., 0., 1., 0.], 
            [0., 0., 0., 1.]]) # next state function

h = matrix([[1., 0., 0., 0.],
            [0., 1., 0., 0]]) # measurement function
r = matrix([[dt, 0.], [0., dt]]) # measurement uncertainty
i = matrix([[1., 0., 0., 0.], [0., 1., 0., 0.], [0., 0., 1., 0.], [0., 0., 0., 1.]]) # identity matrix

In [10]:


def filter(x, p):
    for n in range(len(measurements)):
        
        #prediction
        
        x = f * x + u
        p = f * p * f.transpose()
        
        #measurement update
        z = matrix([measurements[n]])
        y = z.transpose() - h * x
        s = h * p * h.transpose() + r
        k = p * h.transpose() * s.inverse()
        x = x + k * y
        p = (i - k*h)*p
        
        
        print('x= ')
        x.show()
        print('p= ')
        p.show()

filter(x,p)
        

x= 
[4.99009900990099]
[10.01980198019802]
[9.900990099009901]
[-19.801980198019802]
 
p= 
[0.0990099009900991, 0.0, 0.990099009900991, 0.0]
[0.0, 0.0990099009900991, 0.0, 0.990099009900991]
[0.9900990099009874, 0.0, 9.90099009900996, 0.0]
[0.0, 0.9900990099009874, 0.0, 9.90099009900996]
 
x= 
[5.9960079840319365]
[8.007984031936127]
[9.980039920159681]
[-19.960079840319363]
 
p= 
[0.07984031936127742, 0.0, 0.3992015968063879, 0.0]
[0.0, 0.07984031936127742, 0.0, 0.3992015968063879]
[0.39920159680638734, 0.0, 1.9960079840319631, 0.0]
[0.0, 0.39920159680638734, 0.0, 1.9960079840319631]
 
x= 
[6.997858672376874]
[6.004282655246253]
[9.992862241256246]
[-19.98572448251249]
 
p= 
[0.06423982869379022, 0.0, 0.21413276231263487, 0.0]
[0.0, 0.06423982869379022, 0.0, 0.21413276231263487]
[0.21413276231263462, 0.0, 0.7137758743754614, 0.0]
[0.0, 0.21413276231263462, 0.0, 0.7137758743754614]
 
x= 
[7.998667110963012]
[4.002665778073975]
[9.99666777740753]
[-19.99333555481506]
 
p= 
[0.0533155614